# Do Nothing Until You Have a Test

As covered in the "prereqs" notebook, Harry says the whole point is to not write code until you've written a test for what you intend to write, and watch it fail:

In [1]:
from selenium import webdriver
# from selenium.webdriver.firefox.options import Options

# options = Options()
# options.binary_location = (
#     r'/mnt/c/Users/DCM0303/AppData/Local/Mozilla Firefox/firefox.exe'
#     # 'wslview'
# )
# browser = webdriver.Firefox(
#     executable_path = (
#         r'/home/dcm0303/miniconda3/envs/test_dev_book/bin/geckodriver'
#     ),
#     options=options
# )
browser = webdriver.Firefox()
browser.get('http://localhost:8000')

assert 'Django' in browser.title

Ok, it took a long while to get to this initial failure.  The issue was associated with my tendency to try to run notebooks from within the WSL2 env on my Windows laptop, and the difficulty invoking Firefox from within WSL2, and pointing it at the Windows installation.  So I tried using the code above, but it doesn't manage to open a new Firefox window.  As of now, it's giving 

`TimeoutException: Message: Connection refused (os error 111)`

i.e., timing out instead of opening a window.  But it was previously different.  I tried creating the same environment in a Windows-native installation and loading it in a VS Code window running that kernel, but had difficulty installing `conda` via `pip` (it requires a GNU compiler, which is difficult to do without admin permissions).  But I eventually found that Miniconda *can* be installed and the installation dir appended to the Windows path from an installer if you remove the preexisting Python installation in Windows.  From there I was able to reconstitute the environment and load on Windows; I'll hope that downstream steps invoking a browser window will function now.

Anyways, Harry says to anticipate this error: a Firefox window opens, but can't find any host to which to connect, so the page is empty, and you get a `WebDriverException`.

Also, I put it in the cell above, but Harry has us write it to a separate `functional_tests.py` file (he doesn't really work with Jupyter in the book), and to ensure compatability with the text, I'll try to mirror his project dir structure.

In [8]:
import os

proj_dir = r'C:\Data\projects\test_driven_development'
os.chdir(proj_dir)

In [10]:
# %load functional_tests.py
f_tests_str = (
    """from selenium import webdriver

browser = webdriver.Firefox()
browser.get('http://localhost:8000')

assert 'Django' in browser.title"""
)
with open('functional_tests.py', 'w') as f:
    f.write(f_tests_str)

## Getting Django Up and Running

Ok, next he has us execute a command built into Django that's meant to set up a bare-bones project dir structure:

In [15]:
# cmd = 'python -m django-admin.py startproject superlists .'
cmd = 'django-admin.py startproject superlists .'
!{cmd}

Access is denied.


Ok, I tried invoking the command from within this notebook via the IPython shell magic, or even from a Command Prompt window within the Windows OS, and they both fail because of file associations.  I had previously told Windows to default to opening `.py` files with Notepad++, but even once I've destroyed that association by following the instructions [here](https://techcult.com/how-to-remove-file-type-associations-in-windows-10/#Option_4_Remove_File_Association_for_a_particular_app_manually), it won't run, because now it finds no program capable of executing that command.  So it pops up a blue sys error window that says:

~~~bash
This app can't run on your PC

To find a version for your PC, check with the software publisher.
~~~

And, once that's closed, the terminal just prints:

`Access is denied.`

So I instead ran it from within WSL2, and it worked, outputting the following:

~~~bash
.
├── 00_prereqs.ipynb
├── 01_django_setup_functional_test.ipynb
├── dummy.py
├── functional_tests.py
├── geckodriver.log
├── manage.py
├── superlists
│   ├── __init__.py
│   ├── settings.py
│   ├── urls.py
│   └── wsgi.py
├── test_dev_book.yml
└── workspace.code-workspace
~~~

Well, anyways, that's the whole `proj_dir` structure; the stuff added by Django from that command is the `superlists` dir and the `manage.py` file.  Harry says the latter is "Django's Swiss army knife", and crucial for a Django project running as expected.

He has us invoke it by passing that file, and `runserver`, to the interpreter.  Doesn't work via shell magic from within the Notebook, but when run in the terminal, it yields:

~~~bash
(test_dev_book) C:\Data\projects\test_driven_development>python manage.py runserver
Performing system checks...

System check identified no issues (0 silenced).

You have 13 unapplied migration(s). Your project may not work properly until you apply the migrations for app(s): admin, auth, contenttypes, sessions.
Run 'python manage.py migrate' to apply them.
January 20, 2022 - 15:48:10
Django version 1.11.25, using settings 'superlists.settings'
Starting development server at http://127.0.0.1:8000/
Quit the server with CTRL-BREAK.
~~~

When I open that URL in a browser tab (it doesn't launch one for me when handled as above), it shows a blank page saying "It worked!  Congratulations on your first Django-powered page."

Next, with that terminal tab *still open* (i.e., still serving that `runserver` command), you open another, activate the same conda env, and run that `python functional_tests.py`, it opens yet another browser tab (showing the same page as opened by that `runserver` command), and the terminal returns you to the prompt, with no `AssertionError`.  Harry says that's the expected behavior.  So basically, a Django server needs to be running in order for our tests to run as configured.

## Git Repo

Harry enjoins us to maintain the project as a Git repo.

In [18]:
cmds = [
    'git init .',
]
for file in (
    'db.sqlite3',
    'geckodriver.log',
    '00_prereqs.ipynb',
    '01_django_setup_functional_test.ipynb',
):
    cmds.append(f"echo {file} >> .gitignore")

for cmd in cmds:
    !{cmd}

Initialized empty Git repository in C:/Data/projects/test_driven_development/.git/


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>


In [19]:
!git add .
!git status

The file will have its original line endings in your working directory
The file will have its original line endings in your working directory
The file will have its original line endings in your working directory
The file will have its original line endings in your working directory
The file will have its original line endings in your working directory
The file will have its original line endings in your working directory


On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   .gitignore
	new file:   .gitignore.swp
	new file:   functional_tests.py
	new file:   manage.py
	new file:   superlists/__init__.py
	new file:   superlists/__pycache__/__init__.cpython-36.pyc
	new file:   superlists/__pycache__/settings.cpython-36.pyc
	new file:   superlists/__pycache__/urls.cpython-36.pyc
	new file:   superlists/__pycache__/wsgi.cpython-36.pyc
	new file:   superlists/settings.py
	new file:   superlists/urls.py
	new file:   superlists/wsgi.py
	new file:   test_dev_book.yml
	new file:   workspace.code-workspace



In [20]:
!git rm -r --cached superlists/__pycache__
!echo "__pycache__" >> .gitignore
!echo "*.pyc" >> .gitignore

rm 'superlists/__pycache__/__init__.cpython-36.pyc'
rm 'superlists/__pycache__/settings.cpython-36.pyc'
rm 'superlists/__pycache__/urls.cpython-36.pyc'
rm 'superlists/__pycache__/wsgi.cpython-36.pyc'


By default, that *only* maintains the repo within your local drive; I think I'll want to pass it to a Github repo (`git remote`), as well.  One option for initializing a Github repo from the `git` API is [here](https://stackoverflow.com/questions/2423777/is-it-possible-to-create-a-remote-repo-on-github-from-the-cli-without-opening-br#comment18746658_10325316) (well, that's a modification to the commented-upon solution).

It's been forever since I used the `git` API directly; usually just managing projects via the VS Code plugin.  Furthermore, I don't know if I've *ever* used it to create a new repo; probably always just logged into the website in the past, created a new repo there, and then used the resulting URL to associate with local tracked projects.

I found out that the CLI for interacting with Github itself is just `gh`.  There was an installation on my WSL2 VM at `/usr/bin/gh`; I have no idea how it got there, and it apparently isn't managed by `apt`.  It might've been installed from source code somehow; it's [available through conda](https://github.com/cli/cli#linux--bsd), but my base conda env didn't claim it.  Instructions I found kept saying to run `gh auth login`, but that returned "`Error: No such command "auth"`" for me.  I found [this note](https://github.com/cli/cli/issues/1742#issuecomment-694372134) saying that `gh` needed updating.  Since I don't know what installed it initially, I didn't know how to disable the existing installation, so I took the super-safe approach of just renaming it to `/usr/bin/_gh`, and proceeded to use Conda to install a newer version, and authenticated with that.

Anyways, I then checked the repo in a browser, and, being empty, they offered some useful hints on how to populate it via the API:

~~~bash
git remote remove origin
git branch -M main
git remote add origin {url_to_repo}
git push -u origin main
~~~

There's still a lot of muscle memory for me to build with both `git` and the `gh` API.  But it's writing for now.